In [2]:
import numpy as np
import torch
import torchvision.transforms as T
from torchvision.datasets import CIFAR10
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm

In [3]:
def get_normalize(features: torch.Tensor):
    means = (features.data/255).mean(axis=(0, 1, 2))
    stdf = (features.data/255).std(axis=(0, 1, 2))
    # x = features.view(n.size(1), -1).mean(1)
    # y = features.view(n.size(1), -1).std(1)
    return means, stdf

In [12]:

train_dataset = CIFAR10(root="../datasets/", train=True, download=True, transform=T.ToTensor())
test_dataset = CIFAR10(root="../datasets/", train=False, download=True, transform=T.ToTensor())

100%|██████████| 170M/170M [00:12<00:00, 13.3MB/s]


Extracting ../datasets/cifar-10-python.tar.gz to ../datasets/
Files already downloaded and verified


In [13]:
means, stds = get_normalize(train_dataset)

In [4]:
def get_augmentations(train: bool = True) -> T.Compose:
    if train:
        train = T.Compose([
            T.Resize((224, 224)),
            T.RandAugment(num_ops = 5, magnitude= 5),
            T.ToTensor(),
            T.Normalize(mean=means, std=stds)]
        )
        return train
    else:
        test = T.Compose([
            T.Resize((224, 224)),
            T.ToTensor(),
            T.Normalize(mean=means, std=stds)]
        )
        return test


In [14]:
def predict(model: nn.Module, loader: DataLoader, device: torch.device):
  model.eval()
  pred = []
  with torch.no_grad():
    for X, Y in loader:
      inputs = X.to(device)
      outputs = model(inputs)
      predicted_classes = torch.argmax(outputs, 1)
      pred.append(predicted_classes.cpu())
  all_predictions = torch.cat(pred, dim=0)

  return all_predictions

In [ ]:
class FirstModel(nn.Module):
    def __init__(self):
        super().__init__()

        # Изменили размер линейного слоя в зависимости от выходного размера
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),  # 224 x 224 x 16
            nn.ReLU(),
            nn.MaxPool2d(2),  # 112 x 112 x 16

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),  # 112 x 112 x 32
            nn.ReLU(),
            nn.MaxPool2d(2),  # 56 x 56 x 32

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),  # 56 x 56 x 64
            nn.ReLU(),
            nn.MaxPool2d(2),  # 28 x 28 x 64

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),  # 28 x 28 x 128
            nn.ReLU(),
            nn.MaxPool2d(2),  # 14 x 14 x 128

            nn.Flatten(),

            # Размер после сверток и пулинга: 14 * 14 * 128 = 25088
            nn.Linear(14 * 14 * 128, 1024),
            nn.ReLU(),
            nn.Linear(1024, 128),
            nn.ReLU(),
            nn.Linear(128, 10)  # 10 классов для CIFAR-10
        )

    def forward(self, x):
        return self.net(x)

In [15]:



def train(model) -> float:
    model.train()

    train_loss = 0
    total = 0
    correct = 0

    for x, y in tqdm(train_loader, desc='Train'):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()

        output = model(x)

        loss = loss_fn(output, y)

        train_loss += loss.item()

        loss.backward()

        optimizer.step()

        _, y_pred = torch.max(output, 1)
        total += y.size(0)
        correct += (y_pred == y).sum().item()

    train_loss /= len(train_loader)
    accuracy = correct / total

    return train_loss, accuracy

In [16]:
#!g1.1
@torch.inference_mode()
def evaluate(model, loader) -> tuple[float, float]:
    model.eval()

    total_loss = 0
    total = 0
    correct = 0

    for x, y in tqdm(loader, desc='Evaluation'):
        x, y = x.to(device), y.to(device)

        output = model(x)

        loss = loss_fn(output, y)

        total_loss += loss.item()

        _, y_pred = torch.max(output, 1)
        total += y.size(0)
        correct += (y_pred == y).sum().item()

    total_loss /= len(loader)
    accuracy = correct / total

    return total_loss, accuracy

In [17]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

cuda:0


In [10]:
train_dataset  = CIFAR10(root="../datasets/", train=True, transform=get_augmentations())
test_dataset = CIFAR10(root="../datasets/", train=True, transform=get_augmentations(False))

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=8, pin_memory=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
loss_fn = nn.CrossEntropyLoss()
model = FirstModel().to(device)

optimizer = Adam(model.parameters(), lr=1e-3)

n_epoch = 10

for epoch in range(n_epoch):
  train_loss, train_accuracy = train(model)
  valid_loss, valid_accuracy = evaluate(model, valid_loader)
  print(f"Epoch [{epoch+1}/{n_epoch}], "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.2f}%")


Evaluation: 100%|██████████| 391/391 [01:26<00:00,  4.52it/s]


Epoch [1/10], Train Loss: 1.7623, Train Accuracy: 0.35%, Valid Loss: 1.3519, Valid Accuracy: 0.51%


Evaluation: 100%|██████████| 391/391 [01:26<00:00,  4.54it/s]


Epoch [2/10], Train Loss: 1.3709, Train Accuracy: 0.51%, Valid Loss: 1.0727, Valid Accuracy: 0.62%


Evaluation: 100%|██████████| 391/391 [01:26<00:00,  4.52it/s]


...

Epoch [8/10], Train Loss: 0.7620, Train Accuracy: 0.73%, Valid Loss: 0.5057, Valid Accuracy: 0.82%


Evaluation: 100%|██████████| 391/391 [01:36<00:00,  4.05it/s]


Epoch [9/10], Train Loss: 0.7099, Train Accuracy: 0.75%, Valid Loss: 0.4613, Valid Accuracy: 0.84%


Evaluation: 100%|██████████| 391/391 [01:26<00:00,  4.52it/s]

Epoch [10/10], Train Loss: 0.6766, Train Accuracy: 0.76%, Valid Loss: 0.4320, Valid Accuracy: 0.85%


In [ ]:
pred = predict(model, valid_loader, device)
test = T.Compose([
          T.Resize((224, 224)),
          T.RandAugment(num_ops = 2, magnitude= 5),
          T.ToTensor(),
          T.Normalize(mean=means, std=stds)]
      )

In [ ]:
pred

tensor([6, 9, 9,  ..., 9, 1, 1])

In [ ]:
test_dataset = CIFAR10(root="../datasets/", train=False, transform=test)
loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=8, pin_memory=True)

In [18]:
def predict_tta(model: nn.Module, loader: DataLoader, device: torch.device, iterations: int = 2):
  model.eval()

  all_logits = []  # Список для хранения логитов

  with torch.no_grad():  # Отключаем градиенты для ускорения вычислений
      for _ in range(iterations):
          iteration_logits = []
          for images, _ in loader:
              images = images.to(device)  # Переносим данные на нужное устройство

              # Прогоняем изображения через модель
              logits = model(images)
              iteration_logits.append(logits)

          # Конкатенируем логиты всех батчей в этой итерации
          all_logits.append(torch.cat(iteration_logits, dim=0))

      # Преобразуем список логитов в тензор размера [N, C, iterations]
      all_logits = torch.stack(all_logits, dim=-1)  # Размер [N, C, iterations]

      # Усредняем по оси iterations (по последней оси)
      avg_logits = all_logits.mean(dim=-1)  # Размер [N, C]

      # Получаем предсказания (по индексу максимального логита)
      preds = torch.argmax(avg_logits, dim=1)  # Индексы классов

  return preds


In [ ]:
predictions = predict_tta(model, loader, device)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
predictions.to('cpu')

tensor([3, 8, 8,  ..., 5, 1, 7])

In [ ]:
len(predictions)

10000

In [ ]:
from sklearn.metrics import accuracy_score
model.eval()
all_labels = []
for inputs, labels in loader:
  all_labels.append(labels.numpy())

all_labels = np.concatenate(all_labels)
all_labels

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


array([3, 8, 8, ..., 5, 1, 7])

In [ ]:
len(all_labels)

10000

In [ ]:
predictions = predictions.to('cpu').numpy()

In [ ]:
predictions

array([3, 8, 8, ..., 5, 1, 7])

In [ ]:
accuracy_score(all_labels, predictions)

0.7505

In [25]:
class SecondPower(nn.Module):
    def __init__(self):
        super().__init__()

        # Изменили размер линейного слоя в зависимости от выходного размера
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),  # 224 x 224 x 16
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 112 x 112 x 16

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),  # 112 x 112 x 32
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 56 x 56 x 32

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),  # 56 x 56 x 64
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 28 x 28 x 64

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),  # 28 x 28 x 128
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 14 x 14 x 128


            nn.Flatten(),

            # Размер после сверток и пулинга: 14 * 14 * 128 = 25088
            nn.Linear(14 * 14 * 128, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            # nn.Dropout(p=0.3),
            nn.Linear(1024, 128),
            nn.ReLU(),
            nn.Linear(128, 10)  # 10 классов для CIFAR-10
        )

    def forward(self, x):
        return self.net(x)

In [22]:
train_transforms = T.Compose(
    [
        T.Resize((224, 224)),
        T.AutoAugment(T.AutoAugmentPolicy.CIFAR10),
        T.ToTensor(),
        T.Normalize(mean=means, std=stds)
    ]
)

test_transforms = T.Compose(
    [
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=means, std=stds)
    ]
)
train_dataset  = CIFAR10(root="../datasets/", train=True, transform=train_transforms)
test_dataset = CIFAR10(root="../datasets/", train=True, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=8, pin_memory=True)

In [26]:
from torch.optim.lr_scheduler import StepLR
loss_fn = nn.CrossEntropyLoss()
model = SecondPower().to(device)

optimizer = Adam(model.parameters(), lr=1e-3)

scheduler = StepLR(optimizer, step_size=25)

n_epoch = 12

for epoch in range(n_epoch):
  train_loss, train_accuracy = train(model)
  valid_loss, valid_accuracy = evaluate(model, valid_loader)
  print(f"Epoch [{epoch+1}/{n_epoch}], "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.2f}%")


Evaluation: 100%|██████████| 391/391 [01:31<00:00,  4.28it/s]


Epoch [1/12], Train Loss: 1.5277, Train Accuracy: 0.45%, Valid Loss: 0.9926, Valid Accuracy: 0.65%


Evaluation: 100%|██████████| 391/391 [01:32<00:00,  4.23it/s]


Epoch [2/12], Train Loss: 1.1688, Train Accuracy: 0.59%, Valid Loss: 0.9064, Valid Accuracy: 0.68%


Evaluation: 100%|██████████| 391/391 [01:29<00:00,  4.38it/s]


...

Epoch [10/12], Train Loss: 0.5989, Train Accuracy: 0.79%, Valid Loss: 0.5007, Valid Accuracy: 0.90%


Evaluation: 100%|██████████| 391/391 [01:30<00:00,  4.34it/s]


Epoch [11/12], Train Loss: 0.5510, Train Accuracy: 0.81%, Valid Loss: 0.3484, Valid Accuracy: 0.92%


Evaluation: 100%|██████████| 391/391 [01:32<00:00,  4.22it/s]

Epoch [12/12], Train Loss: 0.5113, Train Accuracy: 0.82%, Valid Loss: 0.3032, Valid Accuracy: 0.94%
